In [4]:
import pandas as pd
# # # start - read in BTC data # # #
datasource = relevant_words.csv
relevant_words_df = pd.read_csv({}.format(
    datasource), index_col=0)

In [6]:
relevant_words_df.columns

Index(['good', ' stop', 'think', ' price', ' buy', 'take', ' money',
       ' bitcoin', 'BTC', ' long', ' trading', 'trade', ' short', ' market',
       'BitMEX', ' money.1', ' position', 'good.1', ' trader', ' buy.1', 'low',
       ' ETH', ' price.1', 'bullish', ' high', ' close', 'buying', ' spot',
       ' ThinkingBitmex', 'bear', ' level', ' chart', 'range', ' profit',
       ' volume', 'dip', ' bid', ' bottom', 'liquidity', ' hedge', ' top',
       'coin', ' call', ' sell', 'exchange', ' SNX', ' orderDeFi', ' bought',
       ' volatility', 'risk', ' bounce', ' ATH', 'PnL', ' size', ' million',
       'selling', ' quarter', ' portfolio', 'signal', ' bull', ' SOL', 'FTT',
       ' shorting', ' 10k', 'bearish', ' option', ' asset', 'leverage',
       ' open', ' 5k', 'bad'],
      dtype='object')

In [9]:
relevant_words_df.columns.tolist()


['good',
 ' stop',
 'think',
 ' price',
 ' buy',
 'take',
 ' money',
 ' bitcoin',
 'BTC',
 ' long',
 ' trading',
 'trade',
 ' short',
 ' market',
 'BitMEX',
 ' money.1',
 ' position',
 'good.1',
 ' trader',
 ' buy.1',
 'low',
 ' ETH',
 ' price.1',
 'bullish',
 ' high',
 ' close',
 'buying',
 ' spot',
 ' ThinkingBitmex',
 'bear',
 ' level',
 ' chart',
 'range',
 ' profit',
 ' volume',
 'dip',
 ' bid',
 ' bottom',
 'liquidity',
 ' hedge',
 ' top',
 'coin',
 ' call',
 ' sell',
 'exchange',
 ' SNX',
 ' orderDeFi',
 ' bought',
 ' volatility',
 'risk',
 ' bounce',
 ' ATH',
 'PnL',
 ' size',
 ' million',
 'selling',
 ' quarter',
 ' portfolio',
 'signal',
 ' bull',
 ' SOL',
 'FTT',
 ' shorting',
 ' 10k',
 'bearish',
 ' option',
 ' asset',
 'leverage',
 ' open',
 ' 5k',
 'bad']

In [ ]:
good, stop,
        think, price, buy,
        take, money, bitcoin,
        BTC, long, trading,
        trade, short, market,
        BitMEX, money, position,
        good, trader, buy,
        low, ETH, price,
        bullish, high, close,
        buying, spot, ThinkingBitmex,
        bear, level, chart,
        range, profit, volume,
        dip, bid, bottom,
        liquidity, hedge, top,
        coin, call, sell,
        exchange, SNX, order
        DeFi, bought, volatility,
        risk, bounce, ATH,
        PnL, size, million,
        selling, quarter, portfolio,
        signal, bull, SOL,
        FTT, shorting, 10k,
        bearish, option, asset,
        leverage, open, 5k,
        bad

In [30]:
import csv


In [76]:
results=[]
with open('relevant_words.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        results.append(row)

In [77]:
results[0].append("okay")

In [78]:
with open('relevant_words.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(results[0]) 